# Behavioural Patterns

- In creational patterns, we looked at strategies to create stuff when the usual way of defining objects is too tedious or insufficient

- In structural patterns, we looked at how we can put objects together to enable additional behaviour, especially in the face of some constraints (e.g. low memory, no access to objects etc)

- Now, we look at how we can specific patterns in the objects we put together can enable some common demands of software
    - Behavioral design patterns are concerned with algorithms and the assignment of responsibilities between objects

## 1. Chain of Responsibility

### Big Picture

- You have an object that accepts one or more requests
- The object implements a range of handlers that deal with some condition of the request
- These handlers deal with the request sequentially, until it spits out something at the end

- For example, if I have a credit card transaction request coming in, I may have a handler to check if this is fraud, a handler to check if the person has paid his bills, etc.
    - Only after running through all my handlers, will I allow the request to go through

### Example

In [9]:
from abc import ABC, abstractmethod
from typing import Any, Optional

class Handler(ABC):
    ## A handler should know what the next handler will be, it needs to "pass the baton" to the next handler
    @abstractmethod
    def set_next(self, handler: 'Handler') -> 'Handler':
        pass

    ## A handler should be able to `handle` the request
    @abstractmethod
    def handle(self, request) -> Optional[str]:
        pass

class AbstractHandler(Handler):
    _next_handler: Optional[Handler] = None

    def set_next(self, handler: Handler) -> Handler:
        self._next_handler = handler
        return handler

    @abstractmethod
    def handle(self, request: Any) -> str:
        if self._next_handler:
            return self._next_handler.handle(request)
        return None

class MonkeyHandler(AbstractHandler):
    def handle(self, request: Any) -> str:
        if request == "Banana":
            return f"Monkey: I'll eat the {request}"
        else:
            return super().handle(request)

class SquirrelHandler(AbstractHandler):
    def handle(self, request: Any) -> str:
        if request == "Nut":
            return f"Squirrel: I'll eat the {request}"
        else:
            return super().handle(request)

class DogHandler(AbstractHandler):
    def handle(self, request: Any) -> str:
        if request == "MeatBall":
            return f"Dog: I'll eat the {request}"
        else:
            return super().handle(request)

def client_code(handler: Handler) -> None:
    for food in ["Nut", "Banana", "Cup of coffee"]:
        print(f"Client: Who wants a {food}?", end='\n')
        result = handler.handle(food)
        if result:
            print(f"  {result}", end="\n")
        else:
            print(f"  {food} was left untouched.", end="\n")


monkey = MonkeyHandler()
squirrel = SquirrelHandler()
dog = DogHandler()

## Define sequence of handlers
monkey.set_next(squirrel).set_next(dog)

print("Chain: Monkey > Squirrel > Dog")
client_code(monkey)

print("Subchain: Squirrel > Dog")
client_code(squirrel)

Chain: Monkey > Squirrel > Dog
Client: Who wants a Nut?
  Squirrel: I'll eat the Nut
Client: Who wants a Banana?
  Monkey: I'll eat the Banana
Client: Who wants a Cup of coffee?
  Cup of coffee was left untouched.
Subchain: Squirrel > Dog
Client: Who wants a Nut?
  Squirrel: I'll eat the Nut
Client: Who wants a Banana?
  Banana was left untouched.
Client: Who wants a Cup of coffee?
  Cup of coffee was left untouched.


### Discussion

- Chain of Responsibility, Command, Mediator and Observer address various ways of connecting senders and receivers of requests:
    - `Chain of Responsibility` passes a request sequentially along a dynamic chain of potential receivers until one of them handles it.
    - `Command` establishes unidirectional connections between senders and receivers.
    - `Mediator` eliminates direct connections between senders and receivers, forcing them to communicate indirectly via a mediator object.
    - `Observer` lets receivers dynamically subscribe to and unsubscribe from receiving requests.

- This has quite a similar idea to `Decorators`, where you handle a request differently depending on what was passed to you
    - But the difference is that `Chain of Responsibility` can choose to stop passing the request, and each handler is independent
    - Each `Decorator` object is an extension of some base class, and doesn't decide whether or not to pass the request on. It simply does the job if the decorator's conditions are met

### Pros and Cons

- Pros
    - You can control the order of request handling.
    - Single Responsibility Principle. You can decouple classes that invoke operations from classes that perform operations.
    - Open/Closed Principle. You can introduce new handlers into the app without breaking the existing client code.

- Cons
    - Some requests may end up unhandled.

### Use Cases

- Use the Chain of Responsibility pattern when your program is expected to process different kinds of requests in various ways, but the exact types of requests and their sequences are unknown beforehand.

- Use the pattern when it’s essential to execute several handlers in a particular order.

- Use the CoR pattern when the set of handlers and their order are supposed to change at runtime.
    - You can have setters to change some field in the handler classes, so you can reorder the "next" handler on the fly

## 2. Command

### Big Picture

- Suppose you have some code where there are multiple ways to achieve some outcome
    - For example, in writing MS Word, you can copy text with Ctrl+C, OR right-click -> copy, OR via the copy button in the tool bar

- The question arises, where do you put the code?
    - Copying the same code repeatedly in all areas where you want the "Copy" functionality is repetitive and prone to error

- So what's the solution?
    - Separate the user interface code (where the user invokes the command) from the backend code (what actually happens)
    - So in the example above, Ctrl+C will invoke a new object called `SaveCommand`, which does the actual work

### Example

In [3]:
from abc import ABC, abstractmethod

class Command(ABC):
    @abstractmethod
    def execute(self) -> None:
        pass

class SimpleCommand(Command):
    def __init__(self, payload: str) -> None:
        self._payload: str = payload

    def execute(self) -> None:
        print(f"SimpleCommand does its own work: ({self._payload=})", end='\n')

class ComplexCommand(Command):
    def __init__(self, receiver: 'Receiver', a: str, b: str) -> None:
        self._receiver: Receiver = receiver
        self._a: str = a
        self._b: str = b

    def execute(self) -> None:
        print("ComplexCommand delegates work to a receiver:", end="\n")
        self._receiver.do_something(self._a)
        self._receiver.do_something(self._a)
        
class Receiver:
    def do_something(self, a: str) -> None:
        print(f"    Receiver: Working on ({a}.)", end="\n")

    def do_something_else(self, b: str) -> None:
        print(f"    Receiver: Also working on ({b}.)", end="\n")

class Invoker:
    _on_start = None
    _on_finish = None

    def set_on_start(self, command: Command):
        self._on_start = command

    def set_on_finish(self, command: Command):
        self._on_finish = command

    def do_something_important(self) -> None:
        print("Invoker: Executing _on_start if it exists")
        if isinstance(self._on_start, Command):
            self._on_start.execute()

        print('...Invoker is doing something...')

        print("Invoker: Executing _on_finish if it exists")
        if isinstance(self._on_finish, Command):
            self._on_finish.execute()


## Client calls the invoker
invoker = Invoker()
invoker.set_on_start(SimpleCommand("Say Hi!"))
receiver = Receiver()
invoker.set_on_finish(ComplexCommand(receiver, "Send email", "Save report"))

invoker.do_something_important()

Invoker: Executing _on_start if it exists
SimpleCommand does its own work: (self._payload='Say Hi!')
...Invoker is doing something...
Invoker: Executing _on_finish if it exists
ComplexCommand delegates work to a receiver:
    Receiver: Working on (Send email.)
    Receiver: Working on (Send email.)


### Discussion

- The basic idea of `Command` is to make something that COULD be a method into an object, which gives you more flexibility 
- The `Command` object may do the actual work, or it may delegate to another object (called the `Receiver`)
- Either way, the `Command` object still sits between the `Invoker` (the thing that calls the command) and the final outcome, with a possible `Receiver` in the way
- The Invoker can be customised to run different things in different ways, which adds to the flexibility of this pattern

### Pros and Cons

- Pros
    - Single Responsibility Principle. You can decouple classes that invoke operations from classes that perform these operations.
    - Open/Closed Principle. You can introduce new commands into the app without breaking existing client code.
    - You can implement undo/redo.
    - You can implement deferred execution of operations.
    - You can assemble a set of simple commands into a complex one.

- Cons
    -  The code may become more complicated since you’re introducing a whole new layer between senders and receivers.

### Use Cases

- Use the Command pattern when you want to parametrize objects with operations.
    - That is, instead of designing something as a method call, it becomes more flexbile
    - At run time, since the `Command` is an object, you can swap the order of the underlying operations, you can store the objects somewhere etc.
    - For instance, if you are designing software for a keyboard, storing the keystrokes as `Command` objects allows you to easily remap each key to a different command

- Use the Command pattern when you want to queue operations, schedule their execution, or execute them remotely.
    - If each `Command` is an object rather than a method, you can serialise it and write it out, which gives more information about the state of the programme than a simple method

- Use the Command pattern when you want to implement reversible operations.
    - With a commmand pattern, you can record the history of all operations
    - This history is a stack with all executed command objects, and their associated snapshots before run-time as backup
    - Having something like this lets you "undo" by just copying the desired state from the Command object

## 3. Iterator

### Big Picture

- In many composite objects, you need to have the ability to step through all elements within itself
- However, there can be many ways to step through elements 
    - e.g. a graph can have depth first or breadth first search

- To enable stepping through an object (i.e. making an object an iterable), we can convert it into an iterable by creating a separate `Iterator` class
    - That is, we don't define how the iteration is done within the object iself
    - But we make another class whose job is to define the iteration process, and use it to wrap around the object (see example below)
    - `AlphabeticalOrderIterator` iterator (defines the how) is defined separately from the `WordsCollection` iterable (object to iterate over)

### Example

In [8]:
from collections.abc import Iterable, Iterator
from typing import Any


class AlphabeticalOrderIterator(Iterator):
    # _position: int
    # _reverse: bool = False

    def __init__(self, collection: 'WordsCollection', reverse: bool = False) -> None:
        self._collection: WordsCollection = collection
        self._reverse: bool = reverse
        self._position = -1 if reverse else 0

    def __next__(self) -> Any:
        try:
            value = self._collection[self._position] ##init position either to start or end of the collection
            self._position += -1 if self._reverse else 1 ##increment/decrement position depending on direction of iteration
        except IndexError:
            raise StopIteration()

        return value

class WordsCollection(Iterable):

    def __init__(self, collection: list[Any] | None = None) -> None:
        self._collection = collection or []

    def __getitem__(self, index: int) -> Any:
        return self._collection[index]

    def __iter__(self) -> AlphabeticalOrderIterator:
        return AlphabeticalOrderIterator(self)

    def get_reverse_iterator(self) -> AlphabeticalOrderIterator:
        return AlphabeticalOrderIterator(self, True)

    def add_item(self, item: Any) -> None:
        self._collection.append(item)
        self._collection = sorted(self._collection)


collection = WordsCollection()
collection.add_item("Birst")
collection.add_item("Airst")
collection.add_item("Cirst")

print("Straight traversal:")
print("\n".join(collection))
print("")

print("Reverse traversal:")
print("\n".join(collection.get_reverse_iterator()), end="")

Straight traversal:
Airst
Birst
Cirst

Reverse traversal:
Cirst
Birst
Airst

### Discussion

- In Python, creating iterators requires you to understand two concepts
    - Iterable: Something that can be iterated over
        - A string is an iterable
        - A list is an iterable
        - A float is NOT an iterable
    - Iterator: An iterator is any object that helps you iterate over another object
        - A string is an iterable, but NOT an iterator
            - `next('some string')` throws an error
        - BUT you can call `string_iter = iter('some string')`
            - This `string_iter` object will make the string an iterator
        
- `Iterators` implement a `__next__` dunder method, and this is called to iterate through the collection
- `Iterables` implement a `__iter__` dunder method, and this can be called to return an iterator

### Pros and Cons

- Pros:
    - Single Responsibility Principle. You can clean up the client code and the collections by extracting bulky traversal algorithms into separate classes.
    - Open/Closed Principle. You can implement new types of collections and iterators and pass them to existing code without breaking anything.
    - You can iterate over the same collection in parallel because each iterator object contains its own iteration state.
    - For the same reason, you can delay an iteration and continue it when needed.

- Cons
    - Applying the pattern can be an overkill if your app only works with simple collections.
    - Using an iterator may be less efficient than going through elements of some specialized collections directly.

### Use Cases

- Use the Iterator pattern when your collection has a complex data structure under the hood, but you want to hide its complexity from clients (either for convenience or security reasons).

- Use the pattern to reduce duplication of the traversal code across your app.

- Use the Iterator when you want your code to be able to traverse different data structures or when types of these structures are unknown beforehand.

## 4. Mediator

### Big Picture

- Suppose that, upon calling an operation in some object, you want to trigger an operation in another object
- If this pattern repeats multiple time, your code is going to look extremely ugly
- A `Mediator` object lets you prettify your code by encapsulating the "triggering" logic into a single class

- In the example below, the `Component` objects run some operations `do_X`, and the mediator takes care of the follow up operations

### Example

In [16]:
from abc import ABC, abstractmethod
from typing import Optional

class Mediator(ABC):
    @abstractmethod
    def notify(self, sender: object, event: str) -> None:
        ...

class ConcreteMediator(Mediator):
    def __init__(self, component1: 'Component1', component2: 'Component2') -> None:
        self._component1 = component1
        self._component2 = component2
        self._register_components_to_mediator()

    def _register_components_to_mediator(self):
        print(f'Registering {self._component1.__class__.__name__} and {self._component2.__class__.__name__} to mediator')
        self._component1.mediator = self
        self._component2.mediator = self

    def notify(self, sender: object, event: str) -> None:
        print(f'Mediator received notification from {sender.__class__.__name__}')
        if event == "A":
            print("Mediator reacts on A and triggers following operations:")
            self._component2.do_c()
        elif event == "D":
            print("Mediator reacts on D and triggers following operations:")
            self._component1.do_b()
            self._component2.do_c()

class BaseComponent:
    def __init__(self) -> None:
        self._mediator = None

    @property
    def mediator(self) -> Optional[Mediator]:
        return self._mediator

    @mediator.setter
    def mediator(self, mediator: Mediator) -> None:
        self._mediator = mediator

class Component1(BaseComponent):
    def do_a(self) -> None:
        print("Component 1 does A.")
        if self.mediator is not None:
            self.mediator.notify(self, "A")

    def do_b(self) -> None:
        print("Component 1 does B.")
        if self.mediator is not None:
            self.mediator.notify(self, "B")


class Component2(BaseComponent):
    def do_c(self) -> None:
        print("Component 2 does C.")
        if self.mediator is not None:
            self.mediator.notify(self, "C")
        
    def do_d(self) -> None:
        print("Component 2 does D.")
        if self.mediator is not None:
            self.mediator.notify(self, "D")

c1 = Component1()
c2 = Component2()
mediator = ConcreteMediator(c1, c2)

print("Client triggers operation A.")
c1.do_a()

print("\n", end="")

print("Client triggers operation D.")
c2.do_d()

Registering Component1 and Component2 to mediator
Client triggers operation A.
Component 1 does A.
Mediator received notification from Component1
Mediator reacts on A and triggers following operations:
Component 2 does C.
Mediator received notification from Component2

Client triggers operation D.
Component 2 does D.
Mediator received notification from Component2
Mediator reacts on D and triggers following operations:
Component 1 does B.
Mediator received notification from Component1
Component 2 does C.
Mediator received notification from Component2


### Discussion

- Facade and Mediator are quite similar, in that they orchestrate collaboration between lots of tightly coupled classes
    - Both attempt to make life easy
    - But a facade makes life easy by providing a simplified interface, without providing new functionality
        - The subsystem objects do not need to be aware of the facade, and can still continue to communicate directly with each other
    - A Mediator makes itself the center of all communication with all components
        - So this is obviously more distruptive. The components become tightly coupled to the mediator class
    
- Mediator and Observer are quite similar in implementation, let's just treat them as having a different purpose
    - Mediator happens when you want to insert an object between all your other objects in the universe, so that it orchestrates the communication
        - Objects will `notify` the mediator of any changes on their part
    - Observer has the exact same `notify` pattern, BUT it doesn't deal with orchestration
        - Instead, the `notify` is exactly the purpose of the observer pattern; it is used when you want to be told when something happens

### Pros and Cons

- Pros
    - Single Responsibility Principle. You can extract the communications between various components into a single place, making it easier to comprehend and maintain.
    - Open/Closed Principle. You can introduce new mediators without having to change the actual components.
    - You can reduce coupling between various components of a program.
    - You can reuse individual components more easily.

- Cons
    - Over time a mediator can evolve into a God Object.

### Use Cases

- Use the Mediator pattern when it’s hard to change some of the classes because they are tightly coupled to a bunch of other classes.
    - Idea being, future changes to classes can be managed from the Mediator object, rather than needing a rewrite of the classes themselves

- Use the pattern when you can’t reuse a component in a different program because it’s too dependent on other components.
    - Once Mediator is applied, individual components DO NOT need to be aware of other components, and communicate indirectly through the mediator object

- Use the Mediator when you find yourself creating tons of component subclasses just to reuse some basic behavior in various contexts.
    - Since all relationships are contained in the mediator, new behaviour can be controlled via new mediator classes without involving the actual components

## 5. Memento

### Big Picture

- Memento is a behavioral design pattern that lets you save and restore the previous state of an object without revealing the details of its implementation

- There are 3 items in a memento design
    - `Originator`: The class whose state you want to track. Every time this object does something, you generate a new state and save it as a `Memento`
    - `Memento`: The object that stores the state of the originator
    - `Caretaker`: This object is what orchestrates the act of storing Originator states as mementos, and returning Mementos popped from the saved stack

### Example

In [17]:
from abc import ABC, abstractmethod
from datetime import datetime
from random import sample
from string import ascii_letters

class Originator:
    def __init__(self, state: str) -> None:
        self._state = state
        print(f"Originator: My initial state is: {self._state}")

    def do_something(self) -> None:
        print("Originator: I'm doing something important.")
        self._state = self._generate_state_hash(30)
        print(f"Originator: and my state has changed to: {self._state}")

    @staticmethod
    def _generate_state_hash(length: int = 10) -> str:
        return "".join(sample(ascii_letters, length))

    def save(self) -> 'Memento':
        return ConcreteMemento(self._state)

    def restore(self, memento: 'Memento') -> None:
        self._state = memento.get_state()
        print(f"Originator: My state has changed to: {self._state}")


class Memento(ABC):

    @abstractmethod
    def get_name(self) -> str:
        pass

    @abstractmethod
    def get_date(self) -> str:
        pass

    @abstractmethod
    def get_state(self) -> str:
        pass

class ConcreteMemento(Memento):
    def __init__(self, state: str) -> None:
        self._state = state
        self._date = str(datetime.now())[:19]

    def get_state(self) -> str:
        return self._state

    def get_name(self) -> str:
        return f"{self._date} / ({self._state[0:9]}...)"

    def get_date(self) -> str:
        return self._date


class Caretaker:

    def __init__(self, originator: Originator) -> None:
        self._mementos: list[Memento] = []
        self._originator: Originator = originator

    def backup(self) -> None:
        print("Caretaker: Saving Originator's state...", end='\n')
        self._mementos.append(self._originator.save())

    def undo(self) -> None:
        if not len(self._mementos):
            return

        memento = self._mementos.pop()
        print(f"Caretaker: Restoring state to: {memento.get_name()}")
        try:
            self._originator.restore(memento)
        except Exception:
            self.undo()

    def show_history(self) -> None:
        print("Caretaker: Here's the list of mementos:")
        for memento in self._mementos:
            print(memento.get_name())


originator = Originator("Super-duper-super-puper-super.")
caretaker = Caretaker(originator)

caretaker.backup()
originator.do_something()

caretaker.backup()
originator.do_something()

caretaker.backup()
originator.do_something()

print()
caretaker.show_history()

print("\nClient: Now, let's rollback!\n")
caretaker.undo()

print("\nClient: Once more!\n")
caretaker.undo()

Originator: My initial state is: Super-duper-super-puper-super.
Caretaker: Saving Originator's state...
Originator: I'm doing something important.
Originator: and my state has changed to: ZIEopRSbwDgdyYeMNtKiOcLFUaqBnH
Caretaker: Saving Originator's state...
Originator: I'm doing something important.
Originator: and my state has changed to: DqSCmAUpxXwvcBkaGyMdbKhuYrjseP
Caretaker: Saving Originator's state...
Originator: I'm doing something important.
Originator: and my state has changed to: RdhpEbYUaKDmftyiAjXJOclLonQGrv

Caretaker: Here's the list of mementos:
2024-04-15 17:55:31 / (Super-dup...)
2024-04-15 17:55:31 / (ZIEopRSbw...)
2024-04-15 17:55:31 / (DqSCmAUpx...)

Client: Now, let's rollback!

Caretaker: Restoring state to: 2024-04-15 17:55:31 / (DqSCmAUpx...)
Originator: My state has changed to: DqSCmAUpxXwvcBkaGyMdbKhuYrjseP

Client: Once more!

Caretaker: Restoring state to: 2024-04-15 17:55:31 / (ZIEopRSbw...)
Originator: My state has changed to: ZIEopRSbwDgdyYeMNtKiOcLFUa

### Discussion

- You can combine Memento with Command object to implement the Undo
    - So the `Command` object is responsible for performing some operation on the target object, and the `Memento` saves the state before the command is executed
    - That is, `Command` replaces the `do_something` method

- You can combine Memento with Iterator object to traverse the saved history

- If the object is not complicated, instead of saving the state as a Memento, you can simply save it as a `Prototype` and restore it as necessary

### Pros and Cons

- Pros
    - You can produce snapshots of the object’s state without violating its encapsulation.
    - You can simplify the originator’s code by letting the caretaker maintain the history of the originator’s state.

- Cons
    - The app might consume lots of RAM if clients create mementos too often.
    - Caretakers should track the originator’s lifecycle to be able to destroy obsolete mementos.
    - Most dynamic programming languages, such as PHP, Python and JavaScript, can’t guarantee that the state within the memento stays untouched.

### Use Cases

- Use the Memento pattern when you want to produce snapshots of the object’s state to be able to restore a previous state of the object.

## 6. Observer

### Big Picture

- The point of the observer pattern is to elegantly deal with event based code; i.e. when event A happens, object should perform some action

- You have 2 elements for an observer pattern
    - A subject (the thing getting observed)
    - An observer (the thing reacting to changes in the subject)

### Example

In [18]:
from abc import ABC, abstractmethod
from random import randrange
from typing import List

class Subject(ABC):
    _state: int = 0

    @property
    @abstractmethod
    def state(self) -> int:
        return self._state

    @abstractmethod
    def attach(self, observer: 'Observer') -> None:
        pass

    @abstractmethod
    def detach(self, observer: 'Observer') -> None:
        pass

    @abstractmethod
    def notify(self) -> None:
        pass

class ConcreteSubject(Subject):
    _state: int = 0
    _observers: List['Observer'] = []

    @property
    def state(self):
        return self._state
    
    @property
    def observers(self) -> List['Observer']:
        return self._observers

    def attach(self, observer: 'Observer') -> None:
        print("Subject: Attached an observer.", end='\n')
        self._observers.append(observer)

    def detach(self, observer: 'Observer') -> None:
        self._observers.remove(observer)

    def notify(self) -> None:
        print("Subject: Notifying observers...", end='\n')
        for observer in self._observers:
            observer.update(self)

    def some_business_logic(self) -> None:
        print("Subject: I'm doing something important.", end='\n')
        self._state = randrange(0, 10)

        print(f"Subject: My state has just changed to: {self._state}", end='\n')
        self.notify()

class Observer(ABC):
    @abstractmethod
    def update(self, subject: Subject) -> None:
        pass

class ConcreteObserverA(Observer):
    def update(self, subject: Subject) -> None:
        if subject._state < 3:
            print("ConcreteObserverA: Reacted to the event")

class ConcreteObserverB(Observer):
    def update(self, subject: Subject) -> None:
        if subject._state == 0 or subject._state >= 2:
            print("ConcreteObserverB: Reacted to the event")

## Create a subject
subject = ConcreteSubject()

## Create observers
observer_a = ConcreteObserverA()
observer_b = ConcreteObserverB()

## Attach observers to subject
subject.attach(observer_a)
subject.attach(observer_b)

## Subject performs some business logic
subject.some_business_logic()
subject.some_business_logic()

## Detach observer
subject.detach(observer_a)

## Subject performs some business logic again
subject.some_business_logic()

Subject: Attached an observer.
Subject: Attached an observer.
Subject: I'm doing something important.
Subject: My state has just changed to: 7
Subject: Notifying observers...
ConcreteObserverB: Reacted to the event
Subject: I'm doing something important.
Subject: My state has just changed to: 1
Subject: Notifying observers...
ConcreteObserverA: Reacted to the event
Subject: I'm doing something important.
Subject: My state has just changed to: 9
Subject: Notifying observers...
ConcreteObserverB: Reacted to the event


### Discussion

- Quite similar to mediator
    - Mediator elminates mutual dependencies among system components
    - Observer ismore limited, just tries to let objects be "observed" and perform actions when conditions are met

### Pros and Cons

- Pros
    - Open/Closed Principle. You can introduce new subscriber classes without having to change the publisher’s code (and vice versa if there’s a publisher interface).
    - You can establish relations between objects at runtime.

- Cons
    - Subscribers are notified in random order

### Use Cases

- Use the Observer pattern when changes to the state of one object may require changing other objects, and the actual set of objects is unknown beforehand or changes dynamically.

- Use the pattern when some objects in your app must observe others, but only for a limited time or in specific cases.

## 7. State

### Big Picture

- Your programme has some number of possible internal states, and objects need to change their behaviour according to these states

- To implement this pattern, you need
    - `Context` object, the thing that can have different states
        - e.g. a `phone` object can be in on/off state, locked/unlocked state etc
    - `State` object, the state that this `Context` can be in
        - Each `State` implements a commomn, pre-defined interface 

### Example

In [19]:
from abc import ABC, abstractmethod

class Context:
    _state = None

    def __init__(self, state: 'State') -> None:
        self.transition_to(state)

    def transition_to(self, state: 'State'):
        print(f"Context: Transition to {type(state).__name__}")
        self._state = state
        self._state.context = self

    def request1(self):
        self._state.handle_request1()

    def request2(self):
        self._state.handle_request2()


class State(ABC):

    @property
    def context(self) -> Context:
        return self._context

    @context.setter
    def context(self, context: Context) -> None:
        self._context = context

    @abstractmethod
    def handle_request1(self) -> None:
        pass

    @abstractmethod
    def handle_request2(self) -> None:
        pass

class ConcreteStateA(State):
    def handle_request1(self) -> None:
        print("ConcreteStateA handles request1.")
        print("ConcreteStateA wants to change the state of the context.")
        self.context.transition_to(ConcreteStateB())

    def handle_request2(self) -> None:
        print("ConcreteStateA handles request2.")


class ConcreteStateB(State):
    def handle_request1(self) -> None:
        print("ConcreteStateB handles request1.")

    def handle_request2(self) -> None:
        print("ConcreteStateB handles request2.")
        print("ConcreteStateB wants to change the state of the context.")
        self.context.transition_to(ConcreteStateA())


context = Context(ConcreteStateA())
context.request1()
context.request2()

Context: Transition to ConcreteStateA
ConcreteStateA handles request1.
ConcreteStateA wants to change the state of the context.
Context: Transition to ConcreteStateB
ConcreteStateB handles request2.
ConcreteStateB wants to change the state of the context.
Context: Transition to ConcreteStateA


### Discussion

- You can think of this as kind of an extension of `Strategy`
    - Both work with composition (a `Context` object with a `State`)
    - But `Strategy` are independent. They are algorithms that achieve some outcome but in different ways, and don't know about each other. States know about their relationship with other states

### Pros and Cons

- Pros
    - Single Responsibility Principle. Organize the code related to particular states into separate classes.
    - Open/Closed Principle. Introduce new states without changing existing state classes or the context.
    - Simplify the code of the context by eliminating bulky state machine conditionals.

- Cons
    - Applying the pattern can be overkill if a state machine has only a few states or rarely changes.

### Use Cases

- Use the State pattern when you have an object that behaves differently depending on its current state, the number of states is enormous, and the state-specific code changes frequently.

- Use the pattern when you have a class polluted with massive conditionals that alter how the class behaves according to the current values of the class’s fields.

- Use State when you have a lot of duplicate code across similar states and transitions of a condition-based state machine.

## 8. Strategy

### Big Picture

- Kind of like state, except `Strategy` is mutually exclusive

- You have 2 objects
    - `Context`, which accepts a `Strategy` as an attribute
    - `Strategy`, which implements some behaviour

### Example

In [20]:
from abc import ABC, abstractmethod


class Context():

    def __init__(self, strategy: 'Strategy') -> None:
        self._strategy = strategy

    @property
    def strategy(self) -> 'Strategy':
        return self._strategy

    @strategy.setter
    def strategy(self, strategy: 'Strategy') -> None:
        self._strategy = strategy

    def do_some_business_logic(self) -> None:
        print("Context: Sorting data using the strategy (not sure how it'll do it)")
        result: list[str] = self._strategy.do_algorithm(["a", "b", "c", "d", "e"])
        print(",".join(result))


class Strategy(ABC):
    @abstractmethod
    def do_algorithm(self, data: list[str]) -> list[str]:
        ...

class ConcreteStrategyA(Strategy):
    def do_algorithm(self, data: list[str]) -> list[str]:
        return sorted(data)

class ConcreteStrategyB(Strategy):
    def do_algorithm(self, data: list[str]) -> list[str]:
        return list(reversed(sorted(data)))


context = Context(ConcreteStrategyA())
print("Client: Strategy is set to normal sorting.")
context.do_some_business_logic()
print()

print("Client: Strategy is set to reverse sorting.")
context.strategy = ConcreteStrategyB()
context.do_some_business_logic()

Client: Strategy is set to normal sorting.
Context: Sorting data using the strategy (not sure how it'll do it)
a,b,c,d,e

Client: Strategy is set to reverse sorting.
Context: Sorting data using the strategy (not sure how it'll do it)
e,d,c,b,a


### Discussion

- `Command` and `Strategy` may look similar
    - You can use both to parameterize an object with some action.
    - But the point of `Command` is to convert any operation into an object, which gives you more flexibility
    - `Strategy` is simply meant to define specific actions to be taken

- `Decorator` changes the skin of the object, but `Strategy` changes the internals

- `Template Method` is based on inheritance, but `Strategy` is based on composition
    - `Template Method` lets you alter parts of an algorithm by extending specific methods in subclasses. This works at the class level, and is static
    -  `Strategy` lets you alter parts of the object’s behavior by supplying it with different strategies that correspond to that behavior. Strategy works on the object level, letting you switch behaviors at runtime.

- `State` is like an extension of `Strategy`, where the `States` are aware of each other

### Pros and Cons

- Pros
    - You can swap algorithms used inside an object at runtime.
    - You can isolate the implementation details of an algorithm from the code that uses it.
    - You can replace inheritance with composition.
    - Open/Closed Principle. You can introduce new strategies without having to change the context.

- Cons
    - If you only have a couple of algorithms and they rarely change, there’s no real reason to overcomplicate the program with new classes and interfaces that come along with the pattern.
    - Clients must be aware of the differences between strategies to be able to select a proper one.
    - A lot of modern programming languages have functional type support that lets you implement different versions of an algorithm inside a set of anonymous functions. Then you could use these functions exactly as you’d have used the strategy objects, but without bloating your code with extra classes and interfaces.

### Use Cases

- Use the Strategy pattern when you want to use different variants of an algorithm within an object and be able to switch from one algorithm to another during runtime.

- Use the Strategy when you have a lot of similar classes that only differ in the way they execute some behavior.

- Use the pattern to isolate the business logic of a class from the implementation details of algorithms that may not be as important in the context of that logic.

- Use the pattern when your class has a massive conditional statement that switches between different variants of the same algorithm.

## 9. Template Method

### Big Picture

- You have a method that implements operations in some order

- This order can change based on some criteria, so you want an elegant way of changing this rather than using conditionals

- Simply define the template method in the superclass, and label the variable steps as `abstractmethod` to be overwritten in the subclasses

### Example

In [21]:
from abc import ABC, abstractmethod

class AbstractClass(ABC):
    def template_method(self) -> None:

        self.base_operation1()
        self.required_operations1()
        self.base_operation2()
        self.hook1()
        self.required_operations2()
        self.base_operation3()
        self.hook2()

    def base_operation1(self) -> None:
        print("AbstractClass says: I am doing the bulk of the work")

    def base_operation2(self) -> None:
        print("AbstractClass says: But I let subclasses override some operations")

    def base_operation3(self) -> None:
        print("AbstractClass says: But I am doing the bulk of the work anyway")

    @abstractmethod
    def required_operations1(self) -> None:
        pass

    @abstractmethod
    def required_operations2(self) -> None:
        pass

    def hook1(self) -> None:
        pass

    def hook2(self) -> None:
        pass

class ConcreteClass1(AbstractClass):
    def required_operations1(self) -> None:
        print("ConcreteClass1 says: Implemented Operation1")

    def required_operations2(self) -> None:
        print("ConcreteClass1 says: Implemented Operation2")


class ConcreteClass2(AbstractClass):
    def required_operations1(self) -> None:
        print("ConcreteClass2 says: Implemented Operation1")

    def required_operations2(self) -> None:
        print("ConcreteClass2 says: Implemented Operation2")

    def hook1(self) -> None:
        print("ConcreteClass2 says: Overridden Hook1")


def client_code(abstract_class: AbstractClass) -> None:
    abstract_class.template_method()

print("Same client code can work with different subclasses:")
client_code(ConcreteClass1())
print("")

print("Same client code can work with different subclasses:")
client_code(ConcreteClass2())

Same client code can work with different subclasses:
AbstractClass says: I am doing the bulk of the work
ConcreteClass1 says: Implemented Operation1
AbstractClass says: But I let subclasses override some operations
ConcreteClass1 says: Implemented Operation2
AbstractClass says: But I am doing the bulk of the work anyway

Same client code can work with different subclasses:
AbstractClass says: I am doing the bulk of the work
ConcreteClass2 says: Implemented Operation1
AbstractClass says: But I let subclasses override some operations
ConcreteClass2 says: Overridden Hook1
ConcreteClass2 says: Implemented Operation2
AbstractClass says: But I am doing the bulk of the work anyway


### Discussion

- `Factory Method` is a specialization of Template Method. At the same time, a Factory Method may serve as a step in a large Template Method.

- It is kind of similar to `Strategy`, but `Template Method` is based on inheritance rather than composition
    - Also, template method only lets you override specific parts of the original method, while strategy lets you overhaul the entire method

### Pros and Cons

- Pros
    - You can let clients override only certain parts of a large algorithm, making them less affected by changes that happen to other parts of the algorithm.
    - You can pull the duplicate code into a superclass.

- Cons
    - Some clients may be limited by the provided skeleton of an algorithm.
    - You might violate the Liskov Substitution Principle by suppressing a default step implementation via a subclass.
    - Template methods tend to be harder to maintain the more steps they have.

### Use Cases

- Use the Template Method pattern when you want to let clients extend only particular steps of an algorithm, but not the whole algorithm or its structure.

- Use the pattern when you have several classes that contain almost identical algorithms with some minor differences. As a result, you might need to modify all classes when the algorithm changes.

## 10. Visitor

### Big Picture

- Suppose you have a class that you want to extend with some functionality, but you cannot
- The `Visitor` pattern lets you define a visitor class, which will be invoked by the class itself to do some operation

- There are 2 elements:
    - `Component`: The object that will `accept` a visitor, and once accepted, will ask the visitor to do something
    - `Visitor`: The object that will `accept`-ed by a component, and once accepted, will be asked by the component to do something
    - The implementation of the thing to do will be decided by the component

### Example

In [22]:
from abc import ABC, abstractmethod
from typing import List

class Component(ABC):
    @abstractmethod
    def accept(self, visitor: 'Visitor') -> None:
        pass

class ConcreteComponentA(Component):

    def accept(self, visitor: 'Visitor') -> None:
        visitor.visit_concrete_component_a(self)

    def exclusive_method_of_concrete_component_a(self) -> str:
        return "A"


class ConcreteComponentB(Component):
    def accept(self, visitor: 'Visitor'):
        visitor.visit_concrete_component_b(self)

    def special_method_of_concrete_component_b(self) -> str:
        return "B"

class Visitor(ABC):

    @abstractmethod
    def visit_concrete_component_a(self, element: ConcreteComponentA) -> None:
        pass

    @abstractmethod
    def visit_concrete_component_b(self, element: ConcreteComponentB) -> None:
        pass

class ConcreteVisitor1(Visitor):
    def visit_concrete_component_a(self, element: ConcreteComponentA) -> None:
        print(f"{element.exclusive_method_of_concrete_component_a()} + ConcreteVisitor1")

    def visit_concrete_component_b(self, element: ConcreteComponentB) -> None:
        print(f"{element.special_method_of_concrete_component_b()} + ConcreteVisitor1")


class ConcreteVisitor2(Visitor):
    def visit_concrete_component_a(self, element: ConcreteComponentA) -> None:
        print(f"{element.exclusive_method_of_concrete_component_a()} + ConcreteVisitor2")

    def visit_concrete_component_b(self, element: ConcreteComponentB) -> None:
        print(f"{element.special_method_of_concrete_component_b()} + ConcreteVisitor2")


def client_code(components: List[Component], visitor: Visitor) -> None:
    for component in components:
        component.accept(visitor)

components = [ConcreteComponentA(), ConcreteComponentB()]

print("The client code works with all visitors via the base Visitor interface:")
visitor1 = ConcreteVisitor1()
client_code(components, visitor1)

print("It allows the same client code to work with different types of visitors:")
visitor2 = ConcreteVisitor2()
client_code(components, visitor2)

The client code works with all visitors via the base Visitor interface:
A + ConcreteVisitor1
B + ConcreteVisitor1
It allows the same client code to work with different types of visitors:
A + ConcreteVisitor2
B + ConcreteVisitor2


### Discussion

- Somewhat similar to `Command`, because the object can execute operation over many classes

- The point of having this is because some languages don't support overloading of operators, else you could just call the visitor's "do something" method with the appropriate object and have it do the relevnat job

- But even if you can overload operators, it may not always work!
    - https://refactoring.guru/design-patterns/visitor-double-dispatch

### Pros and Cons

- Pros
    - Open/Closed Principle. You can introduce a new behavior that can work with objects of different classes without changing these classes.
    - Single Responsibility Principle. You can move multiple versions of the same behavior into the same class.
    - A visitor object can accumulate some useful information while working with various objects. This might be handy when you want to traverse some complex object structure, such as an object tree, and apply the visitor to each object of this structure.

- Cons
    - You need to update all visitors each time a class gets added to or removed from the element hierarchy.
    - Visitors might lack the necessary access to the private fields and methods of the elements that they’re supposed to work with.

### Use Cases

- Use the Visitor when you need to perform an operation on all elements of a complex object structure (for example, an object tree).

- Use the Visitor to clean up the business logic of auxiliary behaviors.

- Use the pattern when a behavior makes sense only in some classes of a class hierarchy, but not in others.